# <center><font color='black'><u>Scrapping Spotify</u></font></center>

### <center>L'objectif de ce mini projet sera de prendre la data de mes playlists Spotify, et de réussir à télécharger toutes mes musiques sur mon ordinateur sans quitter Python (ou presque)</center>

 Les gains qu'on aura :
 - plaisir : plus aucune pub pendant l'écoute
 - économie : pas d'abonnement premium à payer
 - écologie : moins de stream, moins de conso, logique

### Avant de commencer...
Le scrapping se base sur Chrome, il faut télécharger le driver qui permettra d'y accéder. Rendez-vous sur ce lien pour télécharger le chromedriver.exe :

https://chromedriver.chromium.org/downloads

<b>>>> Il faut que le fichier chromedriver.exe soit dans le MÊME DOSSIER que ce Notebook ;)</b>

Note : 
  - Penser à accepter (ou refuser) les cookies de chaque page internet à la main, ca pourra éviter des soucis dans le code, on sait jamais
  - Prévoir environ 350Mo pour une centaine de musiques
  - Il peut y avoir des clips qui sont plus longs que la musique, ou des extraits qui changent... C'est assez rare je crois, donc soit tu essaies de rajouter 'audio' après le titre de la musique dans le code, soit tu laisses tourner le code et quand tu vois une erreur tu vas la re-télécharger avec ton site de conversion préféré

### Etape 1 : installer et importer les modules nécessaires

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pytube

Note: you may need to restart the kernel to use updated packages.


In [3]:
try:
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    from pytube import YouTube
    from pytube import Playlist
    import os
    import time
except Exception as e:
    print("Oops, this module is missing : {}".format(e))

# <center><font color='red'>Partie Spotify</font></center>

### Etape 2 : aller jusqu'à la page web
Les URLs sont donnés en dur pour plus de facilité :

- Good mood - https://open.spotify.com/playlist/1q2Ql1FogXoHdqHd1MAuMR

- Chillax - https://open.spotify.com/playlist/263NiClLZvQyD3IsZcqC7h

- Thug gentil - https://open.spotify.com/playlist/7lCCYoGI4WIq5GeSNVFKcv

- TSYN - https://open.spotify.com/playlist/7bwFVcvVc3XNa3EL9AFw2k

(ca c'est mes petites playlists, c'est cadeau)

In [4]:
options = webdriver.ChromeOptions()

options.add_argument("--incognito")
#on passe en mode navigation privée

In [5]:
driver = webdriver.Chrome("./chromedriver", options=options )
#shift+tab pour les infos supplémentaires de synthaxe

driver.get("https://open.spotify.com/playlist/1q2Ql1FogXoHdqHd1MAuMR")

Evidemment, il faut remplacer l'URL par le lien de ta playlist ;)
Tu le trouveras en passant par la version web de Spotify !

### Etape 3 : Prendre les noms des musiques de la playlist

Avant tout, on va vider le fichier pour être sûr qu'il ne soit pas surchargé. Pour cela on l'ouvre en mode 'w'

Tu peux mettre le nom du fichier que tu veux, ca le crééra automatiquement dans le même dossier qu'ici, pense juste à le changer partout sinon ca ne marchera pas

In [6]:
file = open('Musiques.txt','w')
file.write('')
file.close()

On le referme et on va l'ouvrir en mode 'a' pour ajouter les fichiers un à un.

In [7]:
file = open('Musiques.txt','a')
# w pour écrire dans le fichier mais uniquement la ligne donnée
# a pour écrire sans effacer le reste ( .append() )
# r pour la lecture uniquement

unset = set()

Pour toutes les musiques dans le gros bloc, on va prendre le nom de la musique et l'artiste, puis on les rentrera dans le fichier sous la syntaxe suivante :

#### <center>Artiste - Musique  • </center>

- Bloc
  - Musiques
    - Nom + Artiste
    
On va commencer par garder de côté les différentes catégories :
 - <u>musiques</u> : <i>e8ea6a219247d88aa936a012f6227b0d-scss bddcb131e9b40fa874148a30368d83f8-scss</i>
 
 - <u>nom</u> : <i>_966e29b71d2654743538480947a479b3-scss standalone-ellipsis-one-line f3fc214b257ae2f1d43d4c594a94497f-scss</i>
 
 - <u>artiste</u> : <i>da0bc4060bb1bdb4abb8e402916af32e-scss standalone-ellipsis-one-line _8a9c5cc886805907de5073b8ebc3acd8-scss</i>
 
 
<b>Attention à ne pas oublier de scroller jusqu'en bas pour avoir toutes les musiques !</b>

PS: le scroll ne fonctionne pas pour le moment... On peut passer cette étape pour le faire manuellement, les instructions sont données plus bas

In [8]:
# 1) on va scroll en bas de la page pour que tout soit affiché
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
time.sleep(1)

# Visiblement le scroll sur la page Spotify ne veut pas fonctionner...

In [9]:
# Scroll trouvé sur internet
variable_y = 1000
driver.execute_script("window.scrollTo(0, {})".format(variable_y))

In [10]:
# Scroll du TD :
i = 0
while i<1000000 :
    driver.execute_script("window.scrollTo(0, {})".format(i))
    i = i + 100000
    time.sleep(2)

In [11]:
#Bottom of page
page = driver.find_element_by_xpath('/html')
driver.execute_script("""arguments[0].scrollTo(0, arguments[0].scrollHeight);""", page)

Si tout est sorti, on va pouvoir commencer à identifier les noms de musiques (ca ne marche pas pour le moment ces 4 cellules, ne pas faire attention)

JUSTE UN TEST : dans un premier temps on va afficher simplement le nom des musiques telles qu'elles seront enregistrées dans le document pour vérifier que tout fonctionne.

In [12]:
musiques = set(driver.find_elements_by_css_selector("div[class='e8ea6a219247d88aa936a012f6227b0d-scss bddcb131e9b40fa874148a30368d83f8-scss']"))
# Cette variable représente le gros bloc qui contient
# tous les noms de musiques.

for son in musiques:
    print(son.find_element_by_css_selector("span[class='_966e29b71d2654743538480947a479b3-scss standalone-ellipsis-one-line f3fc214b257ae2f1d43d4c594a94497f-scss']").text
          + ' - '
          + son.find_element_by_css_selector("div[class='da0bc4060bb1bdb4abb8e402916af32e-scss standalone-ellipsis-one-line _8a9c5cc886805907de5073b8ebc3acd8-scss']").text
          +' • ')
    

Toploader - Dancing in the Moonlight - 2001 Remix • 
Lomepal - Yeux disent • 
Luidji - Gisèle - Part 4 • 
Robbie Williams - Feel • 
Coldplay - The Scientist • 
Moby - In This World • 
Angèle - Flou • 
Luidji - Le remède • 
The Chain Gang Of 1974 - Sleepwalking • 
Masego, FKJ - Tadow • 
Lomepal, Lost - Danse • 
John Lennon - Imagine - Remastered 2010 • 
Grover Washington, Jr., Bill Withers - Just the Two of Us (feat. Bill Withers) • 
Gnarls Barkley - Crazy • 
Lomepal - Trop beau • 
Elton John - Rocket Man (I Think It's Going To Be A Long, Long Time) • 
The xx - On Hold • 
The Beatles - Let It Be - Remastered 2009 • 
The Beatles - Here Comes The Sun - Remastered 2009 • 
Queen - Bohemian Rhapsody - Remastered 2011 • 
Lou Reed - Walk On the Wild Side • 
Pixies - Where Is My Mind? - Remastered • 
David Bowie - Starman - 2012 Remaster • 
Matt Simons, Falco van den Acker, Bob Van Ratingen - Catch & Release - Deepend Remix • 
Bon Entendeur, Mouloudji - L'amour, l'amour, l'amour • 
David Bowie 

On voit qu'il y a des retours à la lignes, c'est dû à la boucle. Dans le fichier de sortie ils n'y seront pas.

### Etape 4 : Exporter un fichier avec les données de chaque musique

<b>/!\ Attention</b> : il n'est pas possible d'obtenir toutes les musiques d'un coup, il faudra scroller manuellement environ 30 musiques plus bas, puis relancer la cellule ci-dessous, et recommencer cette manipulation jusqu'à la fin des musiques. 

Ne pas hésiter à faire plus de fois cette action pour être sûr de tout avoir, on va par la suite supprimer tous les doublons de noms de musiques.

In [20]:
musiques = set(driver.find_elements_by_css_selector("div[class='e8ea6a219247d88aa936a012f6227b0d-scss bddcb131e9b40fa874148a30368d83f8-scss']"))

for son in musiques:
    file.write(son.find_element_by_css_selector("span[class='_966e29b71d2654743538480947a479b3-scss standalone-ellipsis-one-line f3fc214b257ae2f1d43d4c594a94497f-scss']").text
          + ' - '
          + son.find_element_by_css_selector("div[class='da0bc4060bb1bdb4abb8e402916af32e-scss standalone-ellipsis-one-line _8a9c5cc886805907de5073b8ebc3acd8-scss']").text
          +' • ')

Petite cellule de sécurité pour éviter de lancer la cellule suivante par erreur car elle fermera le fichier où tu enregistres tes musiques. Je te laisse continuer à scroller et à exécuter la cellule du dessus petit à petit

Tu as fini de tout scroller ? Nickel, tu peux continuer alors !

In [21]:
file.close()

# <center><font color='red'>Partie Youtube</font></center>

Je vais commencer par reprendre mon fichier en sélectionnant au hasard des noms de musiques tels qu'ils sont écrits dans le fichier. 

Je vais rentrer à la main ces quelques noms dans la barre de recherche Youtube pour voir si je tombe facilement sur les liens associés à ces musiques.

Après essai, ca semble très bien fonctionner !

### Etape 5 : lançons une fenêtre Youtube

Maintenant on va rentrer tous les noms de musiques 1 par 1 sur Youtube et prendre le lien du 1er résultat qui sort.

In [22]:
driver.get('https://www.youtube.com')
# je n'arrive pas à ouvrir de nouvel onglet mais c'est pas bien grave ca...

Patience, attends que la page ait bien chargé

In [23]:
# on accepte les boutons quand on arrive sur Youtube :
element = driver.find_element_by_css_selector("paper-button[class='style-scope yt-button-renderer style-text size-small']")
element.click()

<b> >>> Ici, il faut appuyer sur le bouton "J'accepte" de Youtube parce que je trouve pas comment appuyer dessus automatiquement...</b>

La cellule ci-dessous ne fonctionne pas pour le moment malheureusement

In [25]:
accept = driver.find_element_by_css_selector("form[class='A28uDc']").click()


# je n'arrive pas à appuyer dessus pour le moment
# je vais appuyer à la main

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"form[class='A28uDc']"}
  (Session info: chrome=88.0.4324.190)


### Etape 6 : on va extraire le nom de chaque musique

On va maintenant reprendre notre fichier contenant le nom de toutes les musiques, pour ensuite aller rentrer le nom de chaque musique dans la barre de recherche Youtube, puis on attrapera le lien de chaque musique à ce moment là.

Pour éviter de modifier le document contenant les musiques, on va simplement l'ouvrir en mode lecture 'r'

D'ailleurs, t'as bien mis le même nom de fichier qu'en haut, hein ?

In [26]:
lecture = open('Musiques.txt','r')

In [27]:
names = lecture.read().split('•')

On stocke tous ces noms dans une liste, pour pouvoir les séparer un par un ensuite.

In [28]:
for son in names :
    print(son)

Toploader - Dancing in the Moonlight - 2001 Remix 
 Lomepal - Yeux disent 
 Luidji - Gisèle - Part 4 
 Robbie Williams - Feel 
 Coldplay - The Scientist 
 Moby - In This World 
 Angèle - Flou 
 Luidji - Le remède 
 The Chain Gang Of 1974 - Sleepwalking 
 Masego, FKJ - Tadow 
 Lomepal, Lost - Danse 
 John Lennon - Imagine - Remastered 2010 
 Grover Washington, Jr., Bill Withers - Just the Two of Us (feat. Bill Withers) 
 Gnarls Barkley - Crazy 
 Lomepal - Trop beau 
 Elton John - Rocket Man (I Think It's Going To Be A Long, Long Time) 
 The xx - On Hold 
 The Beatles - Let It Be - Remastered 2009 
 The Beatles - Here Comes The Sun - Remastered 2009 
 Queen - Bohemian Rhapsody - Remastered 2011 
 Lou Reed - Walk On the Wild Side 
 Pixies - Where Is My Mind? - Remastered 
 David Bowie - Starman - 2012 Remaster 
 Matt Simons, Falco van den Acker, Bob Van Ratingen - Catch & Release - Deepend Remix 
 Bon Entendeur, Mouloudji - L'amour, l'amour, l'amour 
 David Bowie - Life on Mars? - 2015 Re

A ce stade, on va supprimer les doublons de musiques pour ne pas perdre de temps, ni d'espace de stockage

In [29]:
names = list(dict.fromkeys(names))
for son in names :
    print(son)

Toploader - Dancing in the Moonlight - 2001 Remix 
 Lomepal - Yeux disent 
 Luidji - Gisèle - Part 4 
 Robbie Williams - Feel 
 Coldplay - The Scientist 
 Moby - In This World 
 Angèle - Flou 
 Luidji - Le remède 
 The Chain Gang Of 1974 - Sleepwalking 
 Masego, FKJ - Tadow 
 Lomepal, Lost - Danse 
 John Lennon - Imagine - Remastered 2010 
 Grover Washington, Jr., Bill Withers - Just the Two of Us (feat. Bill Withers) 
 Gnarls Barkley - Crazy 
 Lomepal - Trop beau 
 Elton John - Rocket Man (I Think It's Going To Be A Long, Long Time) 
 The xx - On Hold 
 The Beatles - Let It Be - Remastered 2009 
 The Beatles - Here Comes The Sun - Remastered 2009 
 Queen - Bohemian Rhapsody - Remastered 2011 
 Lou Reed - Walk On the Wild Side 
 Pixies - Where Is My Mind? - Remastered 
 David Bowie - Starman - 2012 Remaster 
 Matt Simons, Falco van den Acker, Bob Van Ratingen - Catch & Release - Deepend Remix 
 Bon Entendeur, Mouloudji - L'amour, l'amour, l'amour 
 David Bowie - Life on Mars? - 2015 Re

### Etape 7 : on va chercher chaque musique sur Youtube et stocker son lien dans une autre variable

<b>Pour que ca marche, il faut que la barre de recherche de Youtube soit visible !</b>

Si elle ne l'est pas il n'y aura que l'icône de la petite loupe, dans ce cas, étirer la fenêtre manuellement jusqu'à ce que ce soit visible (ou rendre la fenêtre 'inspecter' moins large)

In [30]:
links = []

In [31]:
search_bar = driver.find_element_by_css_selector("input[placeholder='Rechercher']")

In [38]:
for son in names :
    search_bar.send_keys(son) # On entre le nom
    search_bar.send_keys(Keys.ENTER) # On valide la recherche
    time.sleep(1) # on va pas faire paniquer Youtube...
    resultat = driver.find_element_by_css_selector("a[id='video-title']")
    lien = resultat.get_attribute('href')
    links.append(lien)
    time.sleep(1.5) # on va pas faire paniquer Youtube...
    search_bar.clear()

Ca y est maintenant ca va tourner doucement mais sûrement jusqu'à la fin, tu peux lancer toutes les cellules jusqu'à la cellule STOP en bas (tu risques pas de la louper)

In [39]:
links

['https://www.youtube.com/watch?v=lL6LQAOyTX8',
 'https://www.youtube.com/watch?v=FmUDe7P0fzg',
 'https://www.youtube.com/watch?v=GSMYzFOEwNY',
 'https://www.youtube.com/watch?v=iy4mXZN1Zzk',
 'https://www.youtube.com/watch?v=RB-RcX5DS5A',
 'https://www.youtube.com/watch?v=RB-RcX5DS5A',
 'https://www.youtube.com/watch?v=D6Wi3SmmnAE',
 'https://www.youtube.com/watch?v=pjV_pQGxRiM',
 'https://www.youtube.com/watch?v=Q_5z4n5TZ2Q',
 'https://www.youtube.com/watch?v=hC8CH0Z3L54',
 'https://www.youtube.com/watch?v=hC8CH0Z3L54',
 'https://www.youtube.com/watch?v=VOgFZfRVaww',
 'https://www.youtube.com/watch?v=v8oqbWrP1QY',
 'https://www.youtube.com/watch?v=-N4jf6rtyuw',
 'https://www.youtube.com/watch?v=Bkm03KcnKHE',
 'https://www.youtube.com/watch?v=DtVBCG6ThDk',
 'https://www.youtube.com/watch?v=blJKoXWlqJk',
 'https://www.youtube.com/watch?v=QDYfEBY9NM4',
 'https://www.youtube.com/watch?v=KQetemT1sWc',
 'https://www.youtube.com/watch?v=xG16sdjLtc0',
 'https://www.youtube.com/watch?v=oG6fay

On va stocker toutes ces variables de liens dans un fichier histoire de pas les perdre. Si on a un problème dans l'algo on gagnera du temps en les retrouvant directement ici, comme ca on refait pas tout le chargement un peu long de Youtube.

In [ ]:
save = open('Liens.txt','w')
save.write('')
save.close()

In [40]:
save = open('Liens.txt','a')

In [41]:
for element in links:
    save.write(element + '•')

# <center><font color='red'>Partie téléchargement</font></center>

Je suis pas encore sûr que ca fonctionnera bien, si jamais le nombre de téléchargements quotidiens est limité, on va chercher une extension de Python qui permettra de télécharger automatiquement les fichiers MP3.

### Etape 8 : On va utiliser un module Python pour les télécharger

Code original : https://www.geeksforgeeks.org/download-video-in-mp3-format-using-pytube/

<b> >>> PENSER A METTRE UN CHEMIN POUR LE DOSSIER DE TELECHARGEMENT</b>

Sinon, tout va se télécharger dans le dossier actuel

In [42]:
for element in links:
    try:
        url = str(element)
        yt = YouTube(url)
        print(yt.streams.filter(only_audio=True).first())
        video=yt.streams.filter(only_audio=True).first()
        # AJOUTER LE CHEMIN DU DOSSIER OU TU VEUX METTRE TA MUSIQUE :
        out_file = video.download(output_path="")
        base, ext = os.path.splitext(out_file) 
        new_file = base + '.mp3'
        os.rename(out_file, new_file) 
        print(yt.title + " has been successfully downloaded.\n")
    except Exception as e:
        print("Something went wrong : {} \n".format(e))

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Dancing in the Moonlight (2001 Remix) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Lomepal -  Yeux disent (Clip officiel) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Luidji - Gisèle (Part. 4) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Robbie Williams - Feel (Official Video) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Coldplay - The Scientist (Official Video) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" 

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Something went wrong : [WinError 183] Impossible de créer un fichier déjà existant: 'D:\\Musiques\\Chillax\\TCTS - Games (feat K Stewart).mp4' -> 'D:\\Musiques\\Chillax\\TCTS - Games (feat K Stewart).mp3' 

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Luidji - Tu le mérites has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Vista Kicks - Make It Real (Vestige Remix) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Space Oddity 2015 Remastered Version has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Eels - That Look You Give That Guy 1 (Of

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Bob Sinclar - Love Generation (Radio Edit) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Oh Wonder - All We Do has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Something went wrong : [WinError 183] Impossible de créer un fichier déjà existant: 'D:\\Musiques\\Chillax\\Oh Wonder - All We Do.mp4' -> 'D:\\Musiques\\Chillax\\Oh Wonder - All We Do.mp3' 

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Ashe - Moral of the Story feat. Niall Horan (Official Audio) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Foster The People - Pumped up Kic

Something went wrong : [WinError 183] Impossible de créer un fichier déjà existant: 'D:\\Musiques\\Chillax\\Flume & Chet Faker - This Song Is Not About A Girl.mp4' -> 'D:\\Musiques\\Chillax\\Flume & Chet Faker - This Song Is Not About A Girl.mp3' 

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Cassius - I Love U So (Official Video) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
MORIARTY - Jimmy (FD Acoustic session) has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
PÉPITE — Les bateaux has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a.40.2" progressive="False" type="audio">
Reborn has been successfully downloaded.

<Stream: itag="140" mime_type="audio/mp4" abr="128kbps" acodec="mp4a

# <center><font color='red'>STOP</font></center>

### Etape 9 : fermer l'application

In [68]:
driver.quit()